In [1]:
import boto3
from strands.models import BedrockModel
from strands import Agent
from strands_tools import image_reader, file_read
from video_reader import video_reader
from strands.tools import tool


# video_reader is already available as a built-in tool
agent = Agent(tools=[image_reader, file_read, video_reader])

In [2]:
# Updated multimodal system prompt to include video support
MULTIMODAL_SYSTEM_PROMPT = """ You are a helpful assistant that can process documents, images, and videos. 
Analyze their contents and provide relevant information.

You can:

1. For PNG, JPEG/JPG, GIF, or WebP formats use image_reader to process file
2. For PDF, csv, docx, xls or xlsx formats use file_read to process file  
3. For MP4, MOV, AVI, MKV, WebM formats use video_reader to process file
4. Just deliver the answer

When displaying responses:
- Format answers data in a human-readable way
- Highlight important information
- Handle errors appropriately
- Convert technical terms to user-friendly language
- Always reply in the original user language

Always reply in the original user language.
"""


# video_reader is available as built-in tool
# video_reader is already imported above
# load_tool(path="tools/video_reader.py", name="video_reader")  # This was causing errors

session = boto3.Session(region_name='us-west-2')

bedrock_model = BedrockModel(
    #model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    model_id="us.amazon.nova-pro-v1:0",
    boto_session=session,
    streaming=False
)

# Updated multimodal agent with video support
multimodal_agent = Agent(
    system_prompt=MULTIMODAL_SYSTEM_PROMPT,
    tools=[image_reader, file_read, video_reader],
    model=bedrock_model
)

In [3]:
# Ejemplos de uso del agente multimodal completo

# 1. Analizar una imagen
result_image = multimodal_agent("Analiza la imagen Ninas.jpeg y describe lo que ves")
print(result_image)

# 2. Analizar un video
# result_video = multimodal_agent("Analiza el video video.mp4 y explica qué está ocurriendo")
# print(result_video)

# 3. Analizar un documento
# result_doc = multimodal_agent("Resume el contenido del documento i-94-Enrique_mama.pdf")
# print(result_doc)

# 4. Uso directo de las herramientas
# video_analysis = multimodal_agent.tool.video_reader(
#     video_path="video.mp4", 
#     text_prompt="¿Cuáles son los elementos principales en este video?"
# )
# print(video_analysis)

<thinking> Necesito analizar la imagen "Ninas.jpeg" para describir lo que veo en ella. Utilizaré la herramienta "image_reader" para procesar la imagen. </thinking>

Tool #1: image_reader
La imagen muestra a **dos niñas** jugando en un **parque**.

- La niña de la izquierda lleva una chaqueta azul y pantalones azules.
- La niña de la derecha lleva una chaqueta con diseño de alas y pantalones rosas. Ella parece estar lanzando algo, posiblemente una pelota.

El entorno incluye:
- **Césped verde** con algunas flores blancas.
- **Árboles** en el fondo.
- Una **valla** y una **cancha de tenis** al fondo.

Las niñas parecen estar disfrutando de un día al aire libre en el parque.La imagen muestra a **dos niñas** jugando en un **parque**.

- La niña de la izquierda lleva una chaqueta azul y pantalones azules.
- La niña de la derecha lleva una chaqueta con diseño de alas y pantalones rosas. Ella parece estar lanzando algo, posiblemente una pelota.

El entorno incluye:
- **Césped verde** con algu

In [14]:
result_video.message['content'][0]['text']

'El análisis del video "video.mp4" muestra lo siguiente:\n\n- **Escena Principal:** Un niño está sentado en el suelo frente a un televisor.\n- **Comportamiento del Niño:** El niño lleva auriculares y parece estar viendo una caricatura.\n- **Entorno:** El televisor está colocado sobre un mueble de madera, y hay una ventana con persianas en el fondo.\n- **Duración de la Actividad:** El niño permanece sentado durante todo el video, absorto en el contenido de la televisión.\n\n**Detalles Técnicos:**\n- **Modelo Utilizado:** us.amazon.nova-pro-v1:0\n- **Región:** us-west-2\n- **Ruta del Video:** video.mp4\n- **URI de S3:** s3://strands-video-analysis/videos/video.mp4\n\nEl video captura un momento en el que el niño está completamente inmerso en ver una caricatura en la televisión.'

In [6]:
result_video = multimodal_agent("Analiza el video video.mp4 y que se ve en la tele")
print(result_video)

<thinking> Necesito analizar el video "video.mp4" para determinar qué se ve en la televisión. Utilizaré la herramienta "video_reader" para procesar el video con un prompt específico para esta tarea. </thinking> 
Tool #3: video_reader
El análisis del video "video.mp4" muestra lo siguiente:

- **Escena Principal:** Un niño está sentado en el suelo frente a un televisor.
- **Comportamiento del Niño:** El niño lleva auriculares y parece estar viendo una caricatura.
- **Entorno:** El televisor está colocado sobre un mueble de madera, y hay una ventana con persianas en el fondo.
- **Duración de la Actividad:** El niño permanece sentado durante todo el video, absorto en el contenido de la televisión.

**Detalles Técnicos:**
- **Modelo Utilizado:** us.amazon.nova-pro-v1:0
- **Región:** us-west-2
- **Ruta del Video:** video.mp4
- **URI de S3:** s3://strands-video-analysis/videos/video.mp4

El video captura un momento en el que el niño está completamente inmerso en ver una caricatura en la telev

In [ ]:
result_image.message['content'][0]['text']

In [4]:
video_analysis = multimodal_agent.tool.video_reader(
     video_path="video.mp4", 
     text_prompt="¿Cuáles son los elementos principales en este video?"
 )
print(video_analysis)

{'status': 'success', 'content': [{'text': '🎥 Video Analysis (Model: us.amazon.nova-pro-v1:0):\n\nAt the 7.3-second mark, the video shows a child sitting on the floor in front of a television.'}], 'video_path': 'video.mp4', 's3_uri': 's3://strands-video-analysis/videos/video.mp4', 'model_id_used': 'us.amazon.nova-pro-v1:0', 'region_used': 'us-west-2', 'model_response': {'role': 'assistant', 'content': [{'text': 'At the 7.3-second mark, the video shows a child sitting on the floor in front of a television.'}]}, 'toolUseId': 'tooluse_video_reader_934257536'}
